### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [11]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [12]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [13]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [14]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [15]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [16]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [17]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [18]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [19]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [20]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


#### Punto 1

In [21]:
import random

random.seed(0)
random_indices = random.sample(range(X_train.shape[0]), 5)

# Códigos ANSI para colores
green = "\033[92m"
blue = "\033[94m"
yellow = "\033[93m"
red = "\033[91m"
reset = "\033[0m"
bold = "\033[1m"

for idx in random_indices:
    print(f"{bold}{green}Documento {idx}:{reset}")
    print(f"{bold}{blue}Etiqueta: {newsgroups_train.target_names[newsgroups_train.target[idx]]}{reset}\n")
    print(newsgroups_train.data[idx])
    print()

    cossim = cosine_similarity(X_train[idx], X_train)[0]
    most_similar_indices = np.argsort(cossim)[::-1][1:6]

    print(f"{yellow}Documentos más similares:{reset}")
    for similar_idx in most_similar_indices:
        print(f"\n{green}Documento {similar_idx}:{reset}")
        print(f"{blue}Etiqueta: {newsgroups_train.target_names[newsgroups_train.target[similar_idx]]}{reset}")
        print(f"{red}Similaridad: {cossim[similar_idx]}{reset}\n")
        print(newsgroups_train.data[similar_idx])
    
    print(f"\n{red}{'=' * 76}{reset}\n")

Documento 6311:
Etiqueta: comp.graphics


Yes, I did punch in the wrong numbers (working too many late nites).  I
intended on stating 640x400 is 256,000 bytes.  It's not in the bios, just my
VESA TSR.


Documentos más similares:

Documento 9656:
Etiqueta: comp.graphics
Similaridad: 0.319174103122424


Some VESA bios's support this mode (0x100).  And *any* VGA should be able to
support this (640x480 by 256 colors) since it only requires 256,000 bytes.
My 8514/a VESA TSR supports this; it's the only VESA mode by card can support
due to 8514/a restrictions. (A WD/Paradise)


Documento 5757:
Etiqueta: comp.graphics
Similaridad: 0.1843527221836512


	The VESA standard can be requested from VESA:
	VESA
	2150 North First Street, Suite 440
	San Jose, CA 95131-2029

	Ask for the VESA VBE and Super VGA Programming starndards.  VESA
	also defines local bus and other standards.

	The VESA standard only addresses ways in which an application
	can find out info and capabilities of a specific super V

Se puede observar que los 5 documentos más similares de cada uno de los 5 documentos elegidos al azar tienen sentido y en muchos casos tienen la misma etiqueta que el documento que se está comparando.

#### Punto 2

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

param_grid = {
    'tfidf__max_df': [0.7, 0.85, 1.0],
    'tfidf__min_df': [1, 2, 5],
    'clf__alpha': [0.01, 0.1, 0.5, 1.0]
}

for clf in [MultinomialNB(), ComplementNB()]:
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', clf)
    ])

    grid = GridSearchCV(pipeline, param_grid, scoring='f1_macro', cv=5)
    grid.fit(newsgroups_train.data, y_train)

    y_pred = grid.best_estimator_.predict(newsgroups_test.data)

    score = f1_score(y_test, y_pred, average='macro')

    print(f"Mejores parámetros de TfidfVectorizer y {type(clf).__name__}: {grid.best_params_}")
    print(f"Mejor F1-score macro de {type(clf).__name__}: {score}\n")

Mejores parámetros de TfidfVectorizer y MultinomialNB: {'clf__alpha': 0.01, 'tfidf__max_df': 0.7, 'tfidf__min_df': 1}
Mejor F1-score macro de MultinomialNB: 0.6834004173717222

Mejores parámetros de TfidfVectorizer y ComplementNB: {'clf__alpha': 0.1, 'tfidf__max_df': 0.85, 'tfidf__min_df': 1}
Mejor F1-score macro de ComplementNB: 0.6953652590540836



Se puede observar que ComplementNB supera ligeramente a MultinomialNB comparando los F1-score.

#### Punto 3

In [23]:
tfidf_vect = TfidfVectorizer()
tfidfvect.fit(newsgroups_train.data)

X_train_transpose = X_train.T

selected_words = ['argentina', 'religion', 'number', 'virus', 'game']

for word in selected_words:
    print(f"{green}Palabra: {bold}{word}{reset}")

    idx = tfidfvect.vocabulary_[word]
    cossim = cosine_similarity(X_train_transpose[idx], X_train_transpose)[0]
    mostsim = np.argsort(cossim)[::-1][1:6]
    
    print(f"{blue}Palabras más similares:{reset}")
    for similar_idx in mostsim:
        print(f"{idx2word[similar_idx]} - {red}Similaridad: {cossim[similar_idx]}{reset}")
    print()

Palabra: argentina
Palabras más similares:
argentine - Similaridad: 0.994507067598735
simmering - Similaridad: 0.994507067598735
terrors - Similaridad: 0.994507067598735
precautionary - Similaridad: 0.994507067598735
falklands - Similaridad: 0.994507067598735

Palabra: religion
Palabras más similares:
religious - Similaridad: 0.24507616188432185
religions - Similaridad: 0.21164990102133835
categorized - Similaridad: 0.20391512948704682
purpsoe - Similaridad: 0.20075281023003272
crusades - Similaridad: 0.19870797843527094

Palabra: number
Palabras más similares:
dial - Similaridad: 0.22745633204912918
phone - Similaridad: 0.22204212868105475
serial - Similaridad: 0.2046973529713536
the - Similaridad: 0.19389228992976906
of - Similaridad: 0.18874587657286876

Palabra: virus
Palabras más similares:
winhelp - Similaridad: 0.5545442610780614
mwindows - Similaridad: 0.5545442610780614
winos2 - Similaridad: 0.5545442610780614
infects - Similaridad: 0.5487455463300701
antivirus - Similaridad: 

Se puede observar que las palabras más similares de las 5 palabras elegidas en la mayoría de los casos tienen mucha relación y son bastante coherentes por lo que se puede decir que la vectorización está funcionando correctamente.